### 라이브러리 및 데이터 정리

In [64]:
import re
import json
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from sklearn.preprocessing import MinMaxScaler

# company data 가져오기
with open('./data/company.json') as f:
     data = json.loads(f.read())

# 필요한 데이터 딕셔너리 생성
company_loc = {item['id']: {'name':item['name'], 'address':item['location']['address'], 'geo_lat': item['location']['geo_lat'], 'geo_alt': item['location']['geo_alt']} for item in data}

# 회사 데이터 df변환
df = pd.DataFrame.from_dict(company_loc, orient='index')

# df_dong 생성
# 주소에서 '동' 추출
df['dong'] = df['address'].apply(lambda x: re.search(r'\b[가-힣]*동\b', x).group() if re.search(r'\b[가-힣]*동\b', x) else None)

# 동별 개수 세기
df_dong = df['dong'].value_counts().reset_index()

df_dong.columns = ['dong', 'count']

# 동 기준으로 위도와 경도의 평균 계산
df_dong_mean = df.groupby('dong').agg({'geo_lat':'mean', 'geo_alt':'mean'}).reset_index()

# df_dong + 동별 평균 좌표 컬럼
# 동별 개수와 평균 위도, 경도를 결합
df_final = pd.merge(df_dong, df_dong_mean, on='dong')

# # df_final에 스케일러 생성
# scaler = MinMaxScaler(feature_range=(5, 50))
# df_final['scaled_count'] = scaler.fit_transform(df_final[['count']])

In [43]:
company_loc

{'13138': {'name': '프리그로우',
  'address': '부산시 부산진구 양연로4 2층',
  'geo_lat': 35.1751860797821,
  'geo_alt': 129.078568052224},
 '13107': {'name': '비욘드메디슨',
  'address': '서울 강남구 역삼동 824-12 메가시티오피스텔 403호',
  'geo_lat': 37.4973947147549,
  'geo_alt': 127.030232215583},
 '13104': {'name': '언리저브',
  'address': '서울특별시 강남구 논현동 논현로 123길 36',
  'geo_lat': 37.5109939153304,
  'geo_alt': 127.028933358994},
 '13005': {'name': '에스피에이디',
  'address': '서울 성북구 안암로 145, (안암동5가)',
  'geo_lat': 37.5887034223667,
  'geo_alt': 127.031698331241},
 '12939': {'name': '언컷젬스컴퍼니',
  'address': '서울 강남구 강남대로152길 20, (신사동) 5층',
  'geo_lat': 37.5173922520879,
  'geo_alt': 127.021098483702},
 '12675': {'name': '와드',
  'address': '경기 성남시 분당구 대왕판교로 660, (삼평동) 유스페이스1 A동 605호',
  'geo_lat': 37.4004659332759,
  'geo_alt': 127.106812648944},
 '12647': {'name': '메이즈',
  'address': '서울 구로구 디지털로31길 20, (구로동)',
  'geo_lat': 37.4855210244653,
  'geo_alt': 126.894345286511},
 '12644': {'name': '제이에스소프트',
  'address': '서울 영등포구 문래로20

In [44]:
df

,name,address,geo_lat,geo_alt,dong
13138,프리그로우,부산시 부산진구 양연로4 2층,35.175186,129.078568,None
13107,비욘드메디슨,서울 강남구 역삼동 824-12 메가시티오피스텔 403호,37.497395,127.030232,역삼동
13104,언리저브,서울특별시 강남구 논현동 논현로 123길 36,37.510994,127.028933,논현동
13005,에스피에이디,"서울 성북구 안암로 145, (안암동5가)",37.588703,127.031698,None
12939,언컷젬스컴퍼니,"서울 강남구 강남대로152길 20, (신사동) 5층",37.517392,127.021098,신사동
...,...,...,...,...,...
8063,유핏,"서울 중랑구 신내역로3길 40-36, (신내동) 신내데시앙플랙스 지식산업센터 A612",37.617600,127.104013,신내동
8055,엑스포럼,"서울 송파구 백제고분로9길 5, (잠실동) STORY2003 빌딩",37.509382,127.079656,잠실동
8038,에스투더블유,"경기 성남시 분당구 판교역로192번길 12, (삼평동) 3층",37.397511,127.110736,삼평동
8032,인스타페이,서울시 강남구 영동대로85길 38 진성빌딩,37.506637,127.060580,None


In [65]:
df_final

,dong,count,geo_lat,geo_alt
0,역삼동,18,37.499841,127.034917
1,서초동,8,37.492720,127.021778
2,논현동,7,37.511975,127.030917
3,삼평동,6,37.399634,127.109235
4,삼성동,6,37.508863,127.053641
5,구로동,4,37.483840,126.893792
6,가산동,3,37.475723,126.885064
7,대치동,3,37.505068,127.053876
8,도곡동,3,37.490674,127.036681
9,공덕동,3,37.546086,126.951899


### 회사 위치 표시

In [71]:
# 서울 중심부
m = folium.Map(location=[37.5665, 126.9780], zoom_start=10)

# 회사 위치 버블 표시
for idx, row in df.iterrows():
    folium.CircleMarker(location=[row['geo_lat'], row['geo_alt']],
                        popup=row['name'], # 팝업으로 회사 이름 표시
                        radius=2, # 노드 크기
                        fill=True,
                        color='red').add_to(m)

m.save('map_company.html')
m

### 노드 크기 조절

In [72]:
# 서울 중심부
map = folium.Map(location=[37.5665, 126.9780], zoom_start=11)

# 동 별로 회사의 개수 표시
# MarkerCluster 인스턴스 생성
marker_cluster = MarkerCluster().add_to(map)

# 맵에 회사 위치 추가
for idx, row in df.iterrows():
    # 각 회사의 위도, 경도
    folium.CircleMarker(location=[row['geo_lat'], row['geo_alt']],
                        radius=1,
                        popup=row['name'],
                        fill=True,
                        color='red').add_to(map)

for idx, row in df_final.iterrows():
    # 각 동의 위도, 경도
    folium.Marker(location=[row['geo_lat'], row['geo_alt']], 
                  radius=3, #row['scaled_count']
                  popup=row['dong'],
                  color='black',
                  fill=True,
                  fill_color='black',
                  icon=folium.Icon(color='black', icon='star')).add_to(marker_cluster)

map.save('map_dong.html')
map